# 협업 필터링 추천 시스템

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns


path = "../data/movie-datasets/"
data = pd.read_csv(path + "movies_metadata.csv")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [3]:
data = data[['id', 'genres', 'vote_average', 'vote_count', 'popularity', 'title', 'overview']]

### Bayesian Estimate
$$WR\textrm{(weighted rating)} = R*(\frac{v}{(v+m)})+C*(\frac{m}{(v+m)})$$
  
Where:  
  
R = average for the movie (mean) = (Rating)  
v = number of votes for the movie = (votes)  
m = minimum votes required to be listed in the Top 250 (currently 25,000)  
C = the mean vote across the whole report

In [4]:
print(data['vote_count'].max()) # 100% 지점의 득표수
print(data['vote_count'].min()) # 0%지점의 득표수
print(data['vote_count'].quantile(0.95)) # 상위 5%에 들기 위한 최소 득표수

14075.0
0.0
434.0


In [5]:
cdata = data.copy()
tmp_m = cdata['vote_count'].quantile(0.95)
tmp_data = cdata[cdata['vote_count'] >= tmp_m]

tmp_data.shape

(2274, 7)

In [6]:
tmp_data['vote_average'].describe()

count    2274.000000
mean        6.605893
std         0.785673
min         2.900000
25%         6.100000
50%         6.600000
75%         7.200000
max         9.100000
Name: vote_average, dtype: float64

In [7]:
C = tmp_data['vote_average'].mean()
m = cdata['vote_count'].quantile(0.95)